In [1]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [3]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [4]:
page_id = "77691"
topic = 'rhetoric'

In [5]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [6]:
sectionDATA

{'parse': {'title': 'Figure of speech',
  'pageid': 77691,
  'sections': [{'toclevel': 1,
    'level': '2',
    'line': 'Four rhetorical operations',
    'number': '1',
    'index': '1',
    'fromtitle': 'Figure_of_speech',
    'byteoffset': 1676,
    'anchor': 'Four_rhetorical_operations'},
   {'toclevel': 1,
    'level': '2',
    'line': 'Examples',
    'number': '2',
    'index': '2',
    'fromtitle': 'Figure_of_speech',
    'byteoffset': 4396,
    'anchor': 'Examples'},
   {'toclevel': 2,
    'level': '3',
    'line': 'Schemes',
    'number': '2.1',
    'index': '3',
    'fromtitle': 'Figure_of_speech',
    'byteoffset': 8510,
    'anchor': 'Schemes'},
   {'toclevel': 2,
    'level': '3',
    'line': 'Tropes',
    'number': '2.2',
    'index': '4',
    'fromtitle': 'Figure_of_speech',
    'byteoffset': 21974,
    'anchor': 'Tropes'},
   {'toclevel': 1,
    'level': '2',
    'line': 'See also',
    'number': '3',
    'index': '5',
    'fromtitle': 'Figure_of_speech',
    'byteoffset

In [7]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [8]:
# create section list filtering out non-data sections
filt = [
    #'References',
    'See_also',
    'External_links'
]

incl = [
    'Schemes',
    'Tropes'
]

# sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filt]
sec_list = [sec for sec in sectionDATA["parse"]["sections"] if sec['anchor'] in incl]


len(sec_list)

2

In [ ]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [ ]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [9]:
# make section dict if no nested sections
section_dict = {sec['anchor']:[] for sec in sec_list}

In [33]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

[{'*': 'Figures of speech', 'exists': '', 'ns': 0},
 {'*': 'Accismus', 'exists': '', 'ns': 0},
 {'*': 'Accumulation (figure of speech)', 'exists': '', 'ns': 0},
 {'*': 'Action (narrative)', 'exists': '', 'ns': 0},
 {'*': 'Action fiction', 'exists': '', 'ns': 0},
 {'*': 'Adnomination', 'exists': '', 'ns': 0},
 {'*': 'Adventure fiction', 'exists': '', 'ns': 0},
 {'*': 'Adynaton', 'exists': '', 'ns': 0},
 {'*': 'Allegory', 'exists': '', 'ns': 0},
 {'*': 'Alliteration', 'exists': '', 'ns': 0},
 {'*': 'Allusion', 'exists': '', 'ns': 0},
 {'*': 'Alternate history', 'exists': '', 'ns': 0},
 {'*': 'Anacoenosis', 'exists': '', 'ns': 0},
 {'*': 'Anacoluthon', 'exists': '', 'ns': 0},
 {'*': 'Anadiplosis', 'exists': '', 'ns': 0},
 {'*': 'Analogy', 'exists': '', 'ns': 0},
 {'*': 'Anaphora (rhetoric)', 'exists': '', 'ns': 0},
 {'*': 'Anapodoton', 'exists': '', 'ns': 0},
 {'*': 'Anastrophe', 'exists': '', 'ns': 0},
 {'*': 'Antagonist', 'exists': '', 'ns': 0},
 {'*': 'Antanaclasis', 'exists': '', 'ns'

In [34]:
links = [link['*'] for link in linkDATA["parse"]["links"]]
for i, link in enumerate(links):
    print(i, link)

0 Figures of speech
1 Accismus
2 Accumulation (figure of speech)
3 Action (narrative)
4 Action fiction
5 Adnomination
6 Adventure fiction
7 Adynaton
8 Allegory
9 Alliteration
10 Allusion
11 Alternate history
12 Anacoenosis
13 Anacoluthon
14 Anadiplosis
15 Analogy
16 Anaphora (rhetoric)
17 Anapodoton
18 Anastrophe
19 Antagonist
20 Antanaclasis
21 Anthimeria
22 Anthropomorphism
23 Anti-climax (figure of speech)
24 Antihero
25 Antimetabole
26 Antiphrasis
27 Antithesis
28 Antonomasia
29 Aphorism
30 Aphorismus
31 Apologia
32 Apophasis
33 Aporia
34 Aposiopesis
35 Apostrophe (rhetoric)
36 Apposition
37 Appositive
38 Archaism
39 Archenemy
40 Assonance
41 Asteismus
42 Asyndeton
43 Audience
44 Author
45 Auxesis (figure of speech)
46 Backstory
47 Bathos
48 Burlesque metaphor
49 Cacophony
50 Cantastoria
51 Catachresis
52 Cataphora
53 Character (arts)
54 Character arc
55 Character flaw
56 Characterization
57 Chekhov's gun
58 Chiasmus
59 Chris Baldick
60 Circumlocution
61 City of Westminster
62 Clas

In [ ]:
# visualize link data
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

In [10]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()


In [11]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [13]:
for index, line in enumerate(lines):
    print(index, line)

0 {{short description|Intentional deviation from ordinary language, chosen to produce a rhetorical effect}}
1 {{Redirect|Figures of speech|the hip hop group|Figures of Speech}}
2 [[File:Whitehall MOD 45155526.jpg|thumb|[[Whitehall]] is a road in the [[City of Westminster]], [[London]] used synecdochically to refer to the UK civil service, as many government departments  are nearby.]]
3 
4 A '''figure of speech''' or '''rhetorical figure''' is an intentional deviation from ordinary language, chosen to produce a [[rhetoric|rhetorical]] effect.<ref>{{cite web |url=https://books.google.com/books?id=YWBgAAAAcAAJ&pg=PA113 |title=A Grammar of the English Language ... In a series of familiar lectures, etc |accessdate=2015-12-02 |url-status=live |archiveurl=https://web.archive.org/web/20160508135920/https://books.google.com/books?id=YWBgAAAAcAAJ&pg=PA113 |archivedate=2016-05-08 |last1=Mar |first1=Emanuel del |year=1842 }}</ref> Figures of speech are traditionally classified into ''[[scheme (lin

In [30]:
items = []
descriptions = []
with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[77:291]):
        
        if re.match('\*\s*\[\[', line[0:5]):
            #print("%%%Match%%%")
            #print(line)
            try:
                line = re.sub(r'\<ref.*?\</ref>|\<ref.*\/>', '', line)
                
            except:
                topic_error_line = f"{topic} line sub parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                
            except:
                topic_error_line = f"{topic} item parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                description = re.search(r'\]\]\:*(.*)', line).group(1)
                
                
            except:
                topic_error_line = f"{topic} description parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            items.append(item)
            descriptions.append(description)
        else:
            continue
            #print("%%%No%%%")
            #print(line)

In [39]:
items = [re.sub(r'\:wiktionary\:', '', item) for item in items]
items = [re.sub(r'\|.*', '', item) for item in items]
items

['Accumulation (figure of speech)',
 'adnomination',
 'alliteration',
 'adynaton',
 'anacoluthon',
 'anadiplosis',
 'anaphora (rhetoric)',
 'anastrophe',
 'Anti-climax (figure of speech)',
 'antanaclasis',
 'anthimeria',
 'antimetabole',
 'antirrhesis',
 'epistrophe',
 'antithesis',
 'aphorismus',
 'aposiopesis',
 'apposition',
 'assonance',
 'asteismus',
 'asterismos',
 'asyndeton',
 'cacophony',
 'cataphora',
 'classification (literature & grammar)',
 'chiasmus',
 'climax (figure of speech)',
 'conversion (word formation)',
 'Literary consonance',
 'correlative verse',
 'diacope',
 'asterismos',
 'tmesis',
 'Ellipsis (linguistics)',
 'elision',
 'enallage',
 'enjambment',
 'enthymeme',
 'epanalepsis',
 'epanodos',
 'epistrophe',
 'epizeuxis',
 'euphony',
 'half rhyme',
 'hendiadys',
 'hendiatris',
 'homeoptoton',
 'homographs',
 'homoioteleuton',
 'homonyms',
 'homophones',
 'homeoteleuton',
 'hypallage',
 'hyperbaton',
 'hyperbole',
 'hypozeuxis',
 'hysteron proteron',
 'isocolon',


In [40]:

item_infos = []
for index, title in enumerate(items):
    with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })


#############################################################################################
##################################### edits parse error #####################################
#### 12, antirrhesis, https://en.wikipedia.org/w/index.php?title=antirrhesis&action=info ####
#############################################################################################

###########################################################################################
#################################### edits parse error ####################################
#### 20, asterismos, https://en.wikipedia.org/w/index.php?title=asterismos&action=info ####
###########################################################################################

#################################################################################################################################################
############################################################### edits parse error ##########################

In [41]:
# transfer item_infos to json file
import json
with open(topic + '_infos.json', 'w') as filepath:
    json.dump(item_infos, filepath)

In [ ]:
error_items = [item[topic] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [78]:
with open(f'{topic}_infos.json', 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [79]:
_item_infos = item_infos
len(_item_infos)

198

In [80]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        #item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [81]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [82]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [83]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [84]:
for item in _item_infos:
    #item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [119]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=False)

In [120]:
by_total

[{'rhetoric': 'synesthesia',
  'watchers': 986,
  'redirects': '27',
  'views': 326564,
  'edits': '3,605',
  'view_rank': 0,
  'edit_rank': 124,
  'watcher_rank': 0,
  'total_rank': 0.0},
 {'rhetoric': 'anthropomorphism',
  'watchers': 701,
  'redirects': '51',
  'views': 56314,
  'edits': '1,999',
  'view_rank': 1,
  'edit_rank': 42,
  'watcher_rank': 1,
  'total_rank': 1.0},
 {'rhetoric': 'metaphor',
  'watchers': 481,
  'redirects': '13',
  'views': 33063,
  'edits': '5,030',
  'view_rank': 3,
  'edit_rank': 147,
  'watcher_rank': 5,
  'total_rank': 4.0},
 {'rhetoric': 'neologism',
  'watchers': 579,
  'redirects': '10',
  'views': 31018,
  'edits': '1,646',
  'view_rank': 6,
  'edit_rank': 38,
  'watcher_rank': 2,
  'total_rank': 4.0},
 {'rhetoric': 'satire',
  'watchers': 474,
  'redirects': '27',
  'views': 46563,
  'edits': '4,465',
  'view_rank': 2,
  'edit_rank': 139,
  'watcher_rank': 6,
  'total_rank': 4.0},
 {'rhetoric': 'metonymy',
  'watchers': 540,
  'redirects': '9',
 

In [121]:
dedupe = []
for item in by_total:
    if item[topic] not in dedupe:
        dedupe.append(item[topic])
        

In [122]:

start = 0
limit = 44
top_items = dedupe[start:limit]

In [123]:
with open(f"{topic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in top_items:
        fp.write(item + '\n')

In [116]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=True).head(44)

,edit_rank,edits,redirects,rhetoric,total_rank,view_rank,views,watcher_rank,watchers
190,124,"3,605",27,synesthesia,0.0,0,326564,0,986
99,42,"1,999",51,anthropomorphism,1.0,1,56314,1,701
155,147,"5,030",13,metaphor,4.0,3,33063,5,481
180,139,"4,465",27,satire,4.0,2,46563,6,474
157,38,"1,646",10,neologism,4.0,6,31018,2,579
156,31,"1,281",9,metonymy,5.5,8,29980,3,540
63,120,"3,072",46,onomatopoeia,7.5,4,32190,11,382
138,125,"3,755",22,euphemism,8.0,12,24397,4,525
91,39,"1,707",11,allegory,8.0,7,30022,9,406
159,121,"3,072",46,onomatopoeia,8.5,5,32190,12,382


In [56]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

,edits,redirects,rhetoric,total_rank,view_rank,views,watcher_rank,watchers
190,"3,605",27,synesthesia,197.0,197,326564,197,986
99,"1,999",51,anthropomorphism,196.0,196,56314,196,701
180,"4,465",27,satire,195.0,195,46563,195,474
155,"5,030",13,metaphor,194.0,194,33063,194,481
63,"3,072",46,onomatopoeia,192.0,192,32190,192,382
159,"3,072",46,onomatopoeia,193.0,193,32190,193,382
157,"1,646",10,neologism,191.0,191,31018,191,579
91,"1,707",11,allegory,190.0,190,30022,190,406
156,"1,281",9,metonymy,189.0,189,29980,189,540
177,"2,104",9,proverb,188.0,188,29960,188,192


In [55]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

,edits,redirects,rhetoric,total_rank,view_rank,views,watcher_rank,watchers
190,"3,605",27,synesthesia,197.0,197,326564,197,986
99,"1,999",51,anthropomorphism,196.0,196,56314,196,701
157,"1,646",10,neologism,191.0,191,31018,191,579
156,"1,281",9,metonymy,189.0,189,29980,189,540
138,"3,755",22,euphemism,185.0,185,24397,185,525
155,"5,030",13,metaphor,194.0,194,33063,194,481
180,"4,465",27,satire,195.0,195,46563,195,474
149,"5,446",18,irony,184.0,184,23935,184,467
141,"6,750",23,humour,178.0,178,18621,178,425
91,"1,707",11,allegory,190.0,190,30022,190,406


In [ ]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                section_dict[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(section_dict[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                section_dict[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in section_dict[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })
